In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Input, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.initializers import glorot_uniform, RandomNormal
import random, os, pickle, copy
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
savedir = os.path.join('./save/transfer_learning_cifar5_multiple_instantiations')

C:\Users\JanEirik\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# SET UP DATA

In [2]:
def map_to_range(arr):
    narr = np.zeros_like(arr, dtype=np.int64)
    vals = np.unique(arr)
    for i, v in enumerate(vals):
        mask = (arr == v)
        narr[mask] = i
    return narr

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
input_shape = (32, 32, 3)
num_classes = 5

y_test = y_test.astype(np.int64).squeeze()
y_train = y_train.astype(np.int64).squeeze()

# labels found to give roughly equal classification performance
set_1_test_mask = np.isin(y_test, np.array([0, 3, 4, 6, 7])).squeeze()
set_1_train_mask = np.isin(y_train, np.array([0, 3, 4, 6, 7])).squeeze()

set_2_test_mask = np.isin(y_test, np.array([1, 2, 5, 8, 9])).squeeze()
set_2_train_mask = np.isin(y_train, np.array([1, 2, 5, 8, 9])).squeeze()

set_1_train_labels = y_train[set_1_train_mask]
x_train_1 = x_train[set_1_train_mask]
set_1_test_labels = y_test[set_1_test_mask]
x_test_1 = x_test[set_1_test_mask]
set_2_train_labels = y_train[set_2_train_mask]
x_train_2 = x_train[set_2_train_mask]
set_2_test_labels = y_test[set_2_test_mask]
x_test_2 = x_test[set_2_test_mask]

y_train_1 = keras.utils.to_categorical(map_to_range(set_1_train_labels), num_classes)
y_train_2 = keras.utils.to_categorical(map_to_range(set_2_train_labels), num_classes)
y_test_1 = keras.utils.to_categorical(map_to_range(set_1_test_labels), num_classes)
y_test_2 = keras.utils.to_categorical(map_to_range(set_2_test_labels), num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
datagen1 = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
    )
datagen1.fit(x_train_1)

datagen2= ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
    )
datagen2.fit(x_train_2)

In [5]:
def set_up_model(lr=1e-3, trainable=[True, True, True, True]):
    keras.backend.clear_session()
    inputs = Input(shape=input_shape)
    x = Conv2D(4, kernel_size=(7,7), strides=(1, 1), activation='relu', padding='same', trainable=trainable[0])(inputs)
    x = MaxPooling2D(pool_size=(2,2), strides=(2,2))(x)
    x = Conv2D(8, kernel_size=(3,3), strides=(1, 1), activation='relu', padding='same', trainable=trainable[1])(x)
    x = MaxPooling2D(pool_size=(2,2), strides=(2,2))(x)
    x = Conv2D(16, kernel_size=(3,3), strides=(1, 1), activation='relu', padding='same', trainable=trainable[2])(x)
    x = MaxPooling2D(pool_size=(2,2), strides=(2,2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu', trainable=trainable[3])(x)
    x = Dropout(0.5)(x)
    predictions = Dense(5, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=predictions)
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adam(lr=lr),
                  metrics=['accuracy'])
    return model

# Multiple instantiations of transfer learning

Using first-layer filters trained on dset2 were able to match performance on dset1 compared to first-layer filters trained on dset1, but not the other way around. Doing this procedure 10 times to see if the results are stable.

In [ ]:
epochs = 100
batch_size = 128

for i in range(10):
    ##### DSET1 #####
    # Train model from scratch
    model = set_up_model(trainable=[True, True, True, True])
    ckpt_training = ModelCheckpoint(os.path.join(savedir, 'weights_training_dset1_%d.h5'%i), 
                                    monitor='val_loss', 
                                    verbose=0, 
                                    save_best_only=True, 
                                    save_weights_only=False,
                                    mode='auto', 
                                    period=1
                                   )
    
    history_training_dset1 = model.fit_generator(datagen1.flow(x_train_1, y_train_1, batch_size=batch_size),
                                  epochs=epochs,
                                  verbose=2,
                                  validation_data=(x_test_1, y_test_1),
                                  callbacks=[ckpt_training]
                                  )
    with open(os.path.join(savedir, 'history_training_dset1_%d.pkl'%i), 'wb') as f:
        pickle.dump(history_training_dset1.history, f)
    
    # Freeze first layer and load best performing network state
    model = set_up_model(trainable=[False, True, True, True])
    model.load_weights(os.path.join(savedir, 'weights_training_dset1_%d.h5'%i))
    orig_weights = model.get_weights()
    nweights = copy.deepcopy(orig_weights[:2])
    for w in orig_weights[2:]:
        nweights.append(glorot_uniform()(w.shape).eval(session=keras.backend.get_session()))

   
    # Train on same dset
    model.set_weights(nweights)
    ckpt_transfer = ModelCheckpoint(os.path.join(savedir, 'weights_transfer_dset1_dset1_%d.h5'%i), 
                                    monitor='val_loss', 
                                    verbose=0, 
                                    save_best_only=True, 
                                    save_weights_only=False, 
                                    mode='auto', 
                                    period=1)
                       
    history_transfer_dset1_dset1 = model.fit_generator(datagen1.flow(x_train_1, y_train_1, batch_size=batch_size),
                                                 epochs=epochs,
                                                 verbose=2,
                                                 validation_data=(x_test_1, y_test_1),
                                                 callbacks=[ckpt_transfer]
                                                 )

    with open(os.path.join(savedir, 'history_transfer_dset1_dset1_%d.pkl'%i), 'wb') as f:
        pickle.dump(history_transfer_dset1_dset1.history, f)
        
        
        
    # Train on opposite dset
    model = set_up_model(trainable=[False, True, True, True])
    model.set_weights(nweights)
    ckpt_transfer = ModelCheckpoint(os.path.join(savedir, 'weights_transfer_dset1_dset2_%d.h5'%i), 
                                    monitor='val_loss', 
                                    verbose=0, 
                                    save_best_only=True, 
                                    save_weights_only=False, 
                                    mode='auto', 
                                    period=1)
                       
    history_transfer_dset1_dset2 = model.fit_generator(datagen2.flow(x_train_2, y_train_2, batch_size=batch_size),
                                                 epochs=epochs,
                                                 verbose=2,
                                                 validation_data=(x_test_2, y_test_2),
                                                 callbacks=[ckpt_transfer]
                                                 )

    with open(os.path.join(savedir, 'history_transfer_dset1_dset2_%d.pkl'%i), 'wb') as f:
        pickle.dump(history_transfer_dset1_dset2.history, f)
                       

            
            
            
                       
                       
    #### DSET2 #####
    # Train model from scratch
    model = set_up_model(trainable=[True, True, True, True])
    ckpt_training = ModelCheckpoint(os.path.join(savedir, 'weights_training_dset2_%d.h5'%i), 
                                    monitor='val_loss', 
                                    verbose=0, 
                                    save_best_only=True, 
                                    save_weights_only=False,
                                    mode='auto', 
                                    period=1
                                   )
    
    history_training_dset2 = model.fit_generator(datagen2.flow(x_train_2, y_train_2, batch_size=batch_size),
                                                 epochs=epochs,
                                                 verbose=2,
                                                 validation_data=(x_test_2, y_test_2),
                                                 callbacks=[ckpt_training]
                                                 )
                       
    with open(os.path.join(savedir, 'history_training_dset2_%d.pkl'%i), 'wb') as f:
        pickle.dump(history_training_dset2.history, f)
    
    # Freeze first layer and load best performing network state
    model = set_up_model(trainable=[False, True, True, True])
    model.load_weights(os.path.join(savedir, 'weights_training_dset2_%d.h5'%i))
    orig_weights = model.get_weights()
    nweights = copy.deepcopy(orig_weights[:2])
    for w in orig_weights[2:]:
        nweights.append(glorot_uniform()(w.shape).eval(session=keras.backend.get_session()))
    
    # Train on same dset
    model.set_weights(nweights)
    ckpt_transfer = ModelCheckpoint(os.path.join(savedir, 'weights_transfer_dset2_dset2_%d.h5'%i), 
                                    monitor='val_loss', 
                                    verbose=0, 
                                    save_best_only=True, 
                                    save_weights_only=False, 
                                    mode='auto', 
                                    period=1
                                   )
                       
    history_transfer_dset2_dset2 = model.fit_generator(datagen2.flow(x_train_2, y_train_2, batch_size=batch_size),
                                                 epochs=epochs,
                                                 verbose=2,
                                                 validation_data=(x_test_2, y_test_2),
                                                 callbacks=[ckpt_transfer]
                                                 )

    with open(os.path.join(savedir, 'history_transfer_dset2_dset2_%d.pkl'%i), 'wb') as f:
        pickle.dump(history_transfer_dset2_dset2.history, f)
        
    # Train on opposite dset
    model = set_up_model(trainable=[False, True, True, True])
    model.set_weights(nweights)
    ckpt_transfer = ModelCheckpoint(os.path.join(savedir, 'weights_transfer_dset2_dset1_%d.h5'%i), 
                                    monitor='val_loss', 
                                    verbose=0, 
                                    save_best_only=True, 
                                    save_weights_only=False, 
                                    mode='auto', 
                                    period=1
                                   )
                       
    history_transfer_dset2_dset1 = model.fit_generator(datagen1.flow(x_train_1, y_train_1, batch_size=batch_size),
                                                 epochs=epochs,
                                                 verbose=2,
                                                 validation_data=(x_test_1, y_test_1),
                                                 callbacks=[ckpt_transfer]
                                                 )

    with open(os.path.join(savedir, 'history_transfer_dset2_dset1_%d.pkl'%i), 'wb') as f:
        pickle.dump(history_transfer_dset2_dset1.history, f)

Epoch 1/100
 - 11s - loss: 1.3568 - acc: 0.4249 - val_loss: 1.1614 - val_acc: 0.5492
Epoch 2/100
 - 9s - loss: 1.1896 - acc: 0.5252 - val_loss: 1.0730 - val_acc: 0.5852
Epoch 3/100
 - 9s - loss: 1.1226 - acc: 0.5577 - val_loss: 1.0017 - val_acc: 0.6164
Epoch 4/100
 - 9s - loss: 1.0833 - acc: 0.5751 - val_loss: 0.9608 - val_acc: 0.6242
Epoch 5/100
 - 9s - loss: 1.0585 - acc: 0.5857 - val_loss: 0.9540 - val_acc: 0.6336
Epoch 6/100
 - 9s - loss: 1.0305 - acc: 0.5993 - val_loss: 0.9331 - val_acc: 0.6392
Epoch 7/100
 - 9s - loss: 1.0051 - acc: 0.6091 - val_loss: 0.9069 - val_acc: 0.6384
Epoch 8/100
 - 9s - loss: 0.9919 - acc: 0.6123 - val_loss: 0.8765 - val_acc: 0.6644
Epoch 9/100
 - 9s - loss: 0.9741 - acc: 0.6218 - val_loss: 0.8609 - val_acc: 0.6668
Epoch 10/100
 - 9s - loss: 0.9607 - acc: 0.6259 - val_loss: 0.8547 - val_acc: 0.6662
Epoch 11/100
 - 9s - loss: 0.9446 - acc: 0.6328 - val_loss: 0.8493 - val_acc: 0.6678
Epoch 12/100
 - 9s - loss: 0.9358 - acc: 0.6406 - val_loss: 0.8367 - val_

Epoch 98/100
 - 9s - loss: 0.7107 - acc: 0.7308 - val_loss: 0.6138 - val_acc: 0.7704
Epoch 99/100
 - 9s - loss: 0.7170 - acc: 0.7306 - val_loss: 0.6324 - val_acc: 0.7572
Epoch 100/100
 - 9s - loss: 0.7082 - acc: 0.7333 - val_loss: 0.6102 - val_acc: 0.7688
Epoch 1/100
 - 9s - loss: 1.3969 - acc: 0.4045 - val_loss: 1.1318 - val_acc: 0.5622
Epoch 2/100
 - 9s - loss: 1.1706 - acc: 0.5358 - val_loss: 1.0418 - val_acc: 0.5998
Epoch 3/100
 - 9s - loss: 1.1068 - acc: 0.5644 - val_loss: 0.9814 - val_acc: 0.6226
Epoch 4/100
 - 9s - loss: 1.0521 - acc: 0.5942 - val_loss: 0.9411 - val_acc: 0.6406
Epoch 5/100
 - 9s - loss: 1.0162 - acc: 0.6080 - val_loss: 0.8956 - val_acc: 0.6540
Epoch 6/100
 - 9s - loss: 0.9922 - acc: 0.6149 - val_loss: 0.8843 - val_acc: 0.6562
Epoch 7/100
 - 9s - loss: 0.9685 - acc: 0.6269 - val_loss: 0.8701 - val_acc: 0.6600
Epoch 8/100
 - 9s - loss: 0.9545 - acc: 0.6344 - val_loss: 0.8380 - val_acc: 0.6804
Epoch 9/100
 - 9s - loss: 0.9396 - acc: 0.6368 - val_loss: 0.8330 - val_

Epoch 95/100
 - 9s - loss: 0.7323 - acc: 0.7250 - val_loss: 0.6781 - val_acc: 0.7470
Epoch 96/100
 - 9s - loss: 0.7374 - acc: 0.7257 - val_loss: 0.6729 - val_acc: 0.7440
Epoch 97/100
 - 9s - loss: 0.7335 - acc: 0.7233 - val_loss: 0.6714 - val_acc: 0.7526
Epoch 98/100
 - 9s - loss: 0.7379 - acc: 0.7238 - val_loss: 0.6794 - val_acc: 0.7416
Epoch 99/100
 - 9s - loss: 0.7470 - acc: 0.7201 - val_loss: 0.6763 - val_acc: 0.7434
Epoch 100/100
 - 9s - loss: 0.7387 - acc: 0.7239 - val_loss: 0.6667 - val_acc: 0.7530
Epoch 1/100
 - 9s - loss: 1.3652 - acc: 0.4147 - val_loss: 1.0648 - val_acc: 0.5716
Epoch 2/100
 - 9s - loss: 1.0820 - acc: 0.5547 - val_loss: 0.9208 - val_acc: 0.6208
Epoch 3/100
 - 9s - loss: 0.9932 - acc: 0.5940 - val_loss: 0.9122 - val_acc: 0.6274
Epoch 4/100
 - 9s - loss: 0.9591 - acc: 0.6065 - val_loss: 0.8583 - val_acc: 0.6570
Epoch 5/100
 - 9s - loss: 0.9108 - acc: 0.6334 - val_loss: 0.8087 - val_acc: 0.6734
Epoch 6/100
 - 9s - loss: 0.8905 - acc: 0.6412 - val_loss: 0.8293 - v

Epoch 92/100
 - 9s - loss: 0.6311 - acc: 0.7605 - val_loss: 0.5724 - val_acc: 0.7858
Epoch 93/100
 - 9s - loss: 0.6200 - acc: 0.7635 - val_loss: 0.5580 - val_acc: 0.7882
Epoch 94/100
 - 9s - loss: 0.6210 - acc: 0.7636 - val_loss: 0.5872 - val_acc: 0.7810
Epoch 95/100
 - 9s - loss: 0.6273 - acc: 0.7599 - val_loss: 0.5355 - val_acc: 0.7956
Epoch 96/100
 - 9s - loss: 0.6241 - acc: 0.7641 - val_loss: 0.5727 - val_acc: 0.7848
Epoch 97/100
 - 9s - loss: 0.6289 - acc: 0.7643 - val_loss: 0.5605 - val_acc: 0.7882
Epoch 98/100
 - 9s - loss: 0.6185 - acc: 0.7649 - val_loss: 0.5412 - val_acc: 0.7902
Epoch 99/100
 - 9s - loss: 0.6178 - acc: 0.7650 - val_loss: 0.5728 - val_acc: 0.7816
Epoch 100/100
 - 9s - loss: 0.6210 - acc: 0.7636 - val_loss: 0.5473 - val_acc: 0.7878
Epoch 1/100
 - 9s - loss: 1.2850 - acc: 0.4401 - val_loss: 1.0370 - val_acc: 0.5702
Epoch 2/100
 - 9s - loss: 1.0743 - acc: 0.5545 - val_loss: 0.9801 - val_acc: 0.5914
Epoch 3/100
 - 9s - loss: 1.0227 - acc: 0.5793 - val_loss: 0.9354 

Epoch 89/100
 - 9s - loss: 0.5969 - acc: 0.7739 - val_loss: 0.5243 - val_acc: 0.8058
Epoch 90/100
 - 9s - loss: 0.5939 - acc: 0.7753 - val_loss: 0.5349 - val_acc: 0.7956
Epoch 91/100
 - 9s - loss: 0.5904 - acc: 0.7778 - val_loss: 0.5377 - val_acc: 0.7972
Epoch 92/100
 - 9s - loss: 0.5869 - acc: 0.7756 - val_loss: 0.5495 - val_acc: 0.7910
Epoch 93/100
 - 9s - loss: 0.5874 - acc: 0.7784 - val_loss: 0.5216 - val_acc: 0.7996
Epoch 94/100
 - 9s - loss: 0.5886 - acc: 0.7762 - val_loss: 0.5243 - val_acc: 0.8052
Epoch 95/100
 - 9s - loss: 0.5935 - acc: 0.7746 - val_loss: 0.5199 - val_acc: 0.8026
Epoch 96/100
 - 9s - loss: 0.5900 - acc: 0.7767 - val_loss: 0.5529 - val_acc: 0.7942
Epoch 97/100
 - 9s - loss: 0.5786 - acc: 0.7803 - val_loss: 0.5192 - val_acc: 0.8052
Epoch 98/100
 - 9s - loss: 0.5788 - acc: 0.7823 - val_loss: 0.5686 - val_acc: 0.7884
Epoch 99/100
 - 9s - loss: 0.5785 - acc: 0.7825 - val_loss: 0.6020 - val_acc: 0.7768
Epoch 100/100
 - 9s - loss: 0.5802 - acc: 0.7815 - val_loss: 0.53

...

Epoch 87/100
 - 9s - loss: 0.7199 - acc: 0.7315 - val_loss: 0.6321 - val_acc: 0.7628
Epoch 88/100
 - 9s - loss: 0.7175 - acc: 0.7310 - val_loss: 0.6725 - val_acc: 0.7472
Epoch 89/100
 - 9s - loss: 0.7136 - acc: 0.7340 - val_loss: 0.6234 - val_acc: 0.7658
Epoch 90/100
 - 9s - loss: 0.7141 - acc: 0.7330 - val_loss: 0.6170 - val_acc: 0.7676
Epoch 91/100
 - 9s - loss: 0.7165 - acc: 0.7284 - val_loss: 0.6556 - val_acc: 0.7514
Epoch 92/100
 - 9s - loss: 0.7148 - acc: 0.7302 - val_loss: 0.6426 - val_acc: 0.7654
Epoch 93/100
 - 9s - loss: 0.7124 - acc: 0.7327 - val_loss: 0.6512 - val_acc: 0.7602
Epoch 94/100
 - 9s - loss: 0.7043 - acc: 0.7358 - val_loss: 0.6427 - val_acc: 0.7590
Epoch 95/100
 - 9s - loss: 0.7119 - acc: 0.7341 - val_loss: 0.6333 - val_acc: 0.7612
Epoch 96/100
 - 9s - loss: 0.7161 - acc: 0.7328 - val_loss: 0.6356 - val_acc: 0.7608
Epoch 97/100
 - 9s - loss: 0.7017 - acc: 0.7377 - val_loss: 0.6511 - val_acc: 0.7584
Epoch 98/100
 - 9s - loss: 0.7028 - acc: 0.7394 - val_loss: 0.620

Epoch 84/100
 - 9s - loss: 0.7717 - acc: 0.7089 - val_loss: 0.6986 - val_acc: 0.7466
Epoch 85/100
 - 9s - loss: 0.7779 - acc: 0.7068 - val_loss: 0.6904 - val_acc: 0.7464
Epoch 86/100
 - 9s - loss: 0.7782 - acc: 0.7077 - val_loss: 0.7474 - val_acc: 0.7248
Epoch 87/100
 - 9s - loss: 0.7687 - acc: 0.7122 - val_loss: 0.7103 - val_acc: 0.7382
Epoch 88/100
 - 9s - loss: 0.7663 - acc: 0.7130 - val_loss: 0.6877 - val_acc: 0.7546
Epoch 89/100
 - 9s - loss: 0.7777 - acc: 0.7094 - val_loss: 0.6779 - val_acc: 0.7556
Epoch 90/100
 - 9s - loss: 0.7671 - acc: 0.7125 - val_loss: 0.7386 - val_acc: 0.7274
Epoch 91/100
 - 9s - loss: 0.7716 - acc: 0.7102 - val_loss: 0.6983 - val_acc: 0.7436
Epoch 92/100
 - 9s - loss: 0.7661 - acc: 0.7124 - val_loss: 0.6737 - val_acc: 0.7510
Epoch 93/100
 - 9s - loss: 0.7698 - acc: 0.7078 - val_loss: 0.6811 - val_acc: 0.7514
Epoch 94/100
 - 9s - loss: 0.7703 - acc: 0.7112 - val_loss: 0.6804 - val_acc: 0.7498
Epoch 95/100
 - 9s - loss: 0.7732 - acc: 0.7117 - val_loss: 0.672

Epoch 81/100
 - 9s - loss: 0.6885 - acc: 0.7346 - val_loss: 0.6205 - val_acc: 0.7664
Epoch 82/100
 - 9s - loss: 0.6786 - acc: 0.7371 - val_loss: 0.6556 - val_acc: 0.7482
Epoch 83/100
 - 9s - loss: 0.6935 - acc: 0.7348 - val_loss: 0.6405 - val_acc: 0.7568
Epoch 84/100
 - 9s - loss: 0.6842 - acc: 0.7400 - val_loss: 0.6137 - val_acc: 0.7640
Epoch 85/100
 - 9s - loss: 0.6752 - acc: 0.7416 - val_loss: 0.6211 - val_acc: 0.7592
Epoch 86/100
 - 9s - loss: 0.6767 - acc: 0.7404 - val_loss: 0.6024 - val_acc: 0.7680
Epoch 87/100
 - 9s - loss: 0.6789 - acc: 0.7415 - val_loss: 0.6186 - val_acc: 0.7604
Epoch 88/100
 - 9s - loss: 0.6791 - acc: 0.7420 - val_loss: 0.6311 - val_acc: 0.7548
Epoch 89/100
 - 9s - loss: 0.6752 - acc: 0.7415 - val_loss: 0.5995 - val_acc: 0.7710
Epoch 90/100
 - 9s - loss: 0.6714 - acc: 0.7428 - val_loss: 0.6182 - val_acc: 0.7594
Epoch 91/100
 - 9s - loss: 0.6773 - acc: 0.7397 - val_loss: 0.6378 - val_acc: 0.7608
Epoch 92/100
 - 9s - loss: 0.6689 - acc: 0.7474 - val_loss: 0.609

Epoch 78/100
 - 9s - loss: 0.6177 - acc: 0.7671 - val_loss: 0.5400 - val_acc: 0.7968
Epoch 79/100
 - 9s - loss: 0.6046 - acc: 0.7709 - val_loss: 0.5086 - val_acc: 0.8046
Epoch 80/100
 - 9s - loss: 0.6009 - acc: 0.7667 - val_loss: 0.5208 - val_acc: 0.8020
Epoch 81/100
 - 9s - loss: 0.6071 - acc: 0.7708 - val_loss: 0.5461 - val_acc: 0.7902
Epoch 82/100
 - 9s - loss: 0.6052 - acc: 0.7678 - val_loss: 0.5584 - val_acc: 0.7884
Epoch 83/100
 - 9s - loss: 0.5984 - acc: 0.7735 - val_loss: 0.5186 - val_acc: 0.7954
Epoch 84/100
 - 9s - loss: 0.6015 - acc: 0.7733 - val_loss: 0.5790 - val_acc: 0.7794
Epoch 85/100
 - 9s - loss: 0.6066 - acc: 0.7702 - val_loss: 0.5854 - val_acc: 0.7790
Epoch 86/100
 - 9s - loss: 0.6032 - acc: 0.7714 - val_loss: 0.5500 - val_acc: 0.7836
Epoch 87/100
 - 9s - loss: 0.6013 - acc: 0.7712 - val_loss: 0.5155 - val_acc: 0.8028
Epoch 88/100
 - 9s - loss: 0.6014 - acc: 0.7733 - val_loss: 0.5378 - val_acc: 0.7928
Epoch 89/100
 - 9s - loss: 0.6010 - acc: 0.7711 - val_loss: 0.549

Epoch 75/100
 - 9s - loss: 0.5962 - acc: 0.7776 - val_loss: 0.5725 - val_acc: 0.7914
Epoch 76/100
 - 9s - loss: 0.5963 - acc: 0.7793 - val_loss: 0.5584 - val_acc: 0.7918
Epoch 77/100
 - 9s - loss: 0.5962 - acc: 0.7754 - val_loss: 0.5499 - val_acc: 0.7936
Epoch 78/100
 - 9s - loss: 0.6034 - acc: 0.7725 - val_loss: 0.5807 - val_acc: 0.7838
Epoch 79/100
 - 9s - loss: 0.5970 - acc: 0.7761 - val_loss: 0.5665 - val_acc: 0.7882
Epoch 80/100
 - 9s - loss: 0.5962 - acc: 0.7721 - val_loss: 0.5321 - val_acc: 0.8018
Epoch 81/100
 - 9s - loss: 0.5913 - acc: 0.7768 - val_loss: 0.5851 - val_acc: 0.7846
Epoch 82/100
 - 9s - loss: 0.5999 - acc: 0.7750 - val_loss: 0.5570 - val_acc: 0.7944
Epoch 83/100
 - 9s - loss: 0.5966 - acc: 0.7724 - val_loss: 0.5826 - val_acc: 0.7846
Epoch 84/100
 - 9s - loss: 0.5921 - acc: 0.7774 - val_loss: 0.5331 - val_acc: 0.7978
Epoch 85/100
 - 9s - loss: 0.5920 - acc: 0.7790 - val_loss: 0.5977 - val_acc: 0.7784
Epoch 86/100
 - 9s - loss: 0.5877 - acc: 0.7789 - val_loss: 0.562

In [ ]:
lr_histories = []
epochs = 150
batch_size = 128
lrs = [1e-2, 5e-3, 1e-3, 5e-4, 1e-4, 5e-5, 1e-5, 5e-6, 1e-6]
for i, lr in enumerate(lrs):
    model = set_up_model(lr=lr)
    history = model.fit_generator(datagen1.flow(x_train_1, y_train_1, batch_size=batch_size),
                                  epochs=epochs, verbose=2, validation_data = (x_test_1, y_test_1))
    lr_histories.append(history.history)
with open(os.path.join(savedir, 'lr_histories.pkl'), 'wb') as f:
    pickle.dump(lr_histories, f)

In [ ]:
with open(os.path.join(savedir, 'lr_histories.pkl'), 'rb') as f:
    histories = pickle.load(f)

In [ ]:
lrs = [1e-2, 5e-3, 1e-3, 5e-4, 1e-4, 5e-5, 1e-5, 5e-6, 1e-6]
for i, h in enumerate(histories):
    plt.plot(h['val_loss'], label=lrs[i])
plt.legend()